In [ ]:
import mne
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path
import os
from time import time
from autoreject import AutoReject, Ransac
import psutil
import gc
import json
from mne.preprocessing import ICA

import utils.config as config
from utils.config import DATASETS

%matplotlib qt

qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in ""


In [2]:
### Defining constants and preparing stuff ###

subjects = DATASETS["Jin2019"].subjects
sessions = DATASETS["Jin2019"].sessions
bids_root = os.path.join(DATASETS["Jin2019"].path, "study1/raw/eeg/")
path_events = os.path.join(DATASETS["Jin2019"].path, "study1/raw/beh/new_events/")

df_subject_data = DATASETS["Jin2019"].extra_info["subject_session_df"]

tmin = config.EEG_SETTINGS["EPOCH_START_SEC"]
tmax = tmin + config.EEG_SETTINGS["EPOCH_LENGTH_SEC"]

mapping_128_to_64 = DATASETS["Jin2019"].mapping_channels
mapping_non_eeg = DATASETS["Jin2019"].mapping_non_eeg

In [3]:
def mark_bad_channels(subject_list, bids_root, bad_channels_dict=None):
    """
    Interactively mark bad channels for each subject and session.
    
    Parameters:
        subject_list (list): List of subject IDs.
        data_dir (str): Directory where raw files are stored.
        bad_channels_dict (dict): Existing dictionary to update (default: None).
    
    Returns:
        dict: Dictionary with bad channels for each subject and session.
    """
    if bad_channels_dict is None:
        bad_channels_dict = {}  # Initialize a new dictionary if not provided

    for subject in subject_list:
        print(f"\nProcessing subject: {subject}")
        
        # Initialize dictionary for the subject if not present
        if subject not in bad_channels_dict:
            bad_channels_dict[subject] = {}
        
        # Loop through sessions (assume session files follow a pattern)
        for session in [1, 2]:  # Replace with actual session names if different
            try:
                # Construct file path
                raw_file = bids_root + df_subject_data.iloc[subject - 1, session - 1] + ".bdf"
                
                # Load raw data
                raw = mne.io.read_raw_bdf(raw_file, preload=True)
                print(f"Loaded: {raw_file}")
                
                # Extracting the channel names
                old_ch_names = raw.ch_names

                ### Temporary channel names ###

                # Placeholder names for the old channels
                temp_ch_names = ['temp_' + ch for ch in old_ch_names[:-9]]
                temp_ch_names.extend(old_ch_names[-9:])
                mapping_old_to_temp = dict(zip(old_ch_names, temp_ch_names))

                # Rename the channels in the dataset
                raw.rename_channels(mapping_old_to_temp)
                raw.rename_channels(mapping_128_to_64)
                raw.rename_channels(mapping_non_eeg)

                # Set the channel types for the EXG channels
                raw.set_channel_types({
                    'sacc_EOG1': 'eog',
                    'sacc_EOG2': 'eog',
                    'blink_EOG1': 'eog',
                    'blink_EOG2': 'eog',
                    'EXG5': 'misc',  # Could be a mastoid, set as misc
                    'EXG6': 'misc',  # Could be a mastoid, set as misc
                    'EXG7': 'misc',  # Could be a mastoid, set as misc
                    'EXG8': 'misc'   # Could be a mastoid, set as misc
                })

                # Identify non-EEG channels
                non_eeg_channels = [ch for ch, ch_type in zip(raw.ch_names, raw.get_channel_types()) if ch_type != 'eeg']

                # Get a list of the channels you want to retain (the new 64-channel names and the non-EEG channels)
                channels_to_keep = list(mapping_128_to_64.values()) + non_eeg_channels

                # Drop the channels not in the 64-channel system
                raw.pick(channels_to_keep)

                # Plot raw data to interactively select bad channels
                print("Mark bad channels interactively and close the plot window to continue...")
                raw.plot(highpass=1, lowpass=40, block=True)
                
                # Save the marked bad channels in the dictionary
                bad_channels_dict[subject][session] = raw.info['bads']
                print(f"Bad channels for subject {subject}, {session}: {raw.info['bads']}")
                del raw
                gc.collect()
            
            except FileNotFoundError:
                print(f"File not found: {raw_file}. Skipping this session.")
                continue

    return bad_channels_dict

In [4]:
### Marking bad channels ###

bad_channels_dict = mark_bad_channels(subjects, bids_root)


Processing subject: 1
Extracting EDF parameters from /home/sivert/Documents/Master_AttentionalDirectionResearch/data/datasets/jin2019/study1/raw/eeg/sub1_1.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 3631615  =      0.000 ...  7092.998 secs...
Loaded: /home/sivert/Documents/Master_AttentionalDirectionResearch/data/datasets/jin2019/study1/raw/eeg/sub1_1.bdf


/tmp/ipykernel_1572708/4070257559.py:49: RuntimeWarning: The unit for channel(s) EXG5, EXG6, EXG7, EXG8 has changed from V to NA.
  raw.set_channel_types({


Mark bad channels interactively and close the plot window to continue...
Setting up band-pass filter from 1 - 40 Hz

IIR filter parameters
---------------------
Butterworth bandpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 16 (effective, after forward-backward)
- Cutoffs at 1.00, 40.00 Hz: -6.02, -6.02 dB

Using matplotlib as 2D backend.
Channels marked as bad:
['FC1', 'C1']
Bad channels for subject 1, 1: ['FC1', 'C1']
Extracting EDF parameters from /home/sivert/Documents/Master_AttentionalDirectionResearch/data/datasets/jin2019/study1/raw/eeg/sub1_2.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 3763199  =      0.000 ...  7349.998 secs...
Loaded: /home/sivert/Documents/Master_AttentionalDirectionResearch/data/datasets/jin2019/study1/raw/eeg/sub1_2.bdf


/tmp/ipykernel_1572708/4070257559.py:49: RuntimeWarning: The unit for channel(s) EXG5, EXG6, EXG7, EXG8 has changed from V to NA.
  raw.set_channel_types({


Mark bad channels interactively and close the plot window to continue...
Setting up band-pass filter from 1 - 40 Hz

IIR filter parameters
---------------------
Butterworth bandpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 16 (effective, after forward-backward)
- Cutoffs at 1.00, 40.00 Hz: -6.02, -6.02 dB

Channels marked as bad:
none
Bad channels for subject 1, 2: []

Processing subject: 2
Extracting EDF parameters from /home/sivert/Documents/Master_AttentionalDirectionResearch/data/datasets/jin2019/study1/raw/eeg/sub2_1.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 3269631  =      0.000 ...  6385.998 secs...
Loaded: /home/sivert/Documents/Master_AttentionalDirectionResearch/data/datasets/jin2019/study1/raw/eeg/sub2_1.bdf


/tmp/ipykernel_1572708/4070257559.py:49: RuntimeWarning: The unit for channel(s) EXG5, EXG6, EXG7, EXG8 has changed from V to NA.
  raw.set_channel_types({


Mark bad channels interactively and close the plot window to continue...
Setting up band-pass filter from 1 - 40 Hz

IIR filter parameters
---------------------
Butterworth bandpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 16 (effective, after forward-backward)
- Cutoffs at 1.00, 40.00 Hz: -6.02, -6.02 dB

Channels marked as bad:
none


KeyboardInterrupt: 

QSocketNotifier: Invalid socket 87 and type 'Read', disabling...


In [ ]:
# Save the bad channels dictionary
bad_channels_file = "jin2019_bad_channels.json"
bad_channels_path = os.path.join(config.LOGS_PATH, "preprocessing_logs/", bad_channels_file)
with open(bad_channels_path, "w") as file:
    json.dump(bad_channels_dict, file)

print(f"Bad channels saved to: {bad_channels_path}")
